In [2]:
from bs4 import BeautifulSoup
import re
import unicodedata

In [3]:

#THIS IS PROBLEMATIC "weeadaffa26. Dezember 1997 bldalsdladad" it doesn't match! 

In [4]:
with open('Corona_Virus_Bielefeld.html',encoding='utf-8') as f:
    raw_txt = f.read()


In [5]:
html = BeautifulSoup(raw_txt)

In [6]:
divs = html.find_all("div","mb-5 bg-weiss p-3 views-row")

In [7]:
divs = [BeautifulSoup(str(div))  for div in divs]

In [8]:
for i,div in enumerate(divs):
    all_p = div.find_all("p")
    for p in all_p:
        if p.text == "Aktuell infektiös":
            p.text


In [9]:
filtered_divs = divs[:167]
p_texts = []
for div in filtered_divs:
    all_p = div.find_all("p")
    for p in all_p:
        print(p.text)
    print("\n")


Informationen zum Coronavirus
Positiv
Aktuell infektiös
Meldeinzidenz
Genesen
Verstorben
Das RKI hat heute drei weitere Sterbefälle gemeldet:
Seit Beginn der Pandemie verzeichnet Bielefeld damit 416 Todesfälle im Zusammenhang mit Corona.
Krankenhaus


Informationen zum Coronavirus
Hinweis zum Melderückstau
Der Melderückstau im Gesundheitsamt ist weitestgehend abgebaut. Die „reale“ Inzidenz – also die Inzidenz mit Nachmeldungen, die durch verspätete Labormeldungen, durch den Meldezeitpunkt um etwa 17 Uhr und durch den zeitweiligen Meldestau entstehen – ist deutlich gesunken. Am 28. November lag die reale Inzidenz, also mit den Nachmeldungen, bei 469,6; am Montag, 6. Dezember, lag sie noch bei 366,4. Die Inzidenz, wie das RKI sie jeden Morgen meldet, lag heute bei 302,8. Es werden zwar im Laufe der nächsten Tage wegen der verspäteten Labormeldungen und wegen des Meldezeitpunkts noch Nachmeldungen dazu kommen, die RKI-Inzidenz wird aber morgen voraussichtlich mit etwa 318 deutlich unter 3

In [10]:
def get_p2li(all_ps):
    p2li = {}
    for p in all_ps:
        try:
            if p.text.strip() in ["Positiv","Aktuell infektiös","Meldeinzidenz","Genesen"]:
                p2li.update({p.text.strip():p.find_next("li").text})
        except AttributeError:
            print("passing because p.text wasn't found")
            pass
    return p2li

In [11]:
def date_2_xml_date(date_str):
    NUM_MONTH = {"Januar":"01","Februar":"02","März":"03","April":"04","Mai":"05","Juni":"06","Juli":"07","August":"08","September":"09","Oktober":"10","November":"11","Dezember":"12"}
    split = date_str.split()
    day = split[0]
    month = split[1]
    year = split[2]

    day = day.replace(".","")
    if len(day) == 1:
        day = "0"+ day
    month = NUM_MONTH[month]

    return year + "-" + month + "-" + day



In [35]:
date_regex = re.compile(r"[0-9]*\..*[0-9][0-9][0-9][0-9]")
date2data = {}
for i,div in enumerate(filtered_divs):
    all_p = div.find_all("p")
    p2li = get_p2li(all_p)
    if list(p2li.keys()) != ["Positiv","Aktuell infektiös","Meldeinzidenz","Genesen"]:
        pass
    else:
    #print(div.find("h3"))
        date_str = div.find("h3").text
        date_str = re.search(date_regex,date_str).group(0)
        date_str = date_2_xml_date(date_str)
        date2data.update({date_str:p2li})


In [201]:
def pos_cleanup(string):
    cleanup_dict = {"eine":"1","vier":"4","fünf":"5"}
    split = string.split()

    sentence = ""
    for word in split:
        if word in cleanup_dict.keys():
            sentence = sentence + " " + cleanup_dict[word]
        else:
            sentence = sentence + " "+ word
    return sentence
    
def find_new_infections(positiv_string):
    positiv_regex = re.compile(r"[0-9]+ (Neuinfektionen|Neuinfektion)")
    cleaned_string = pos_cleanup(positiv_string)
    match = re.search(positiv_regex,cleaned_string).group(0)
    split = match.split()
    return split[0]

In [285]:
def find_est_infecious_people(inf_string):
    infectious_regex = re.compile(r"[0-9]+(\.| )*[0-9]* (Personen|Person)")
    inf_string = unicodedata.normalize("NFKD", inf_string)
    match = re.search(infectious_regex,inf_string).group(0)
    split = match.split()
    if len(split) == 3:
        num_out = split[0]+split[1]
        num_out =num_out.replace(".","")
    else:
        num_out = split[0].replace(".","")
    return num_out

In [286]:
def find_incidence(inc_string):
    inc_regex = re.compile(r"[0-9]+(,[0-9])* pro")
    inc_string = unicodedata.normalize("NFKD", inc_string)
    match = re.search(inc_regex,inc_string).group(0)
    split = match.split()
    num_out = split[0]
    num_out = num_out.replace(",",".")
    return num_out

In [287]:
def find_recovered(rec_string):
    rec_regex = re.compile(r"[0-9]+\.[0-9]*")
    rec_string = unicodedata.normalize("NFKD", rec_string)
    match = re.search(rec_regex,rec_string).group(0)
    num_out = match.replace(".","")
    return num_out


In [288]:
def parse_div(date):
    data = date2data[date]
    pos_string = data["Positiv"]
    inf_string = data["Aktuell infektiös"]
    inc_string = data["Meldeinzidenz"]
    rec_string = data["Genesen"]

    pos_out = find_new_infections(pos_string)
    inf_out = find_est_infecious_people(inf_string)
    inc_out = find_incidence(inc_string)
    rec_out = find_recovered(rec_string)

    return [str(date),pos_out,inf_out,inc_out,rec_out]

In [289]:
 """for key in date2data:
    data = date2data[key]
    string = data["Aktuell infektiös"]
    print(string)
    print(parse_div(key)[2])"""

'for key in date2data:\n   data = date2data[key]\n   string = data["Aktuell infektiös"]\n   print(string)\n   print(parse_div(key)[2])'

In [290]:
complete_data = [parse_div(date) for date in date2data]

In [294]:

months = list(set([data[0].split("-")[1] for data in complete_data]))
month_data = {month:[data for data in complete_data if data[0].split("-")[1] == month] for month in months}

def daydata2xml(datapoint):
    valid_format = """<day_data date="{date}">
        <positiv_gestested>{pos}</positiv_gestested>
        <geschaetzt_infektioes>{est_inf}</geschaetzt_infektioes>
        <inzidenz>{inc}</inzidenz>
        <genesen>{rec}</genesen>
    </day_data>"""
    
    date = datapoint[0]
    pos = datapoint[1]
    est_inf = datapoint[2]
    inc = datapoint[3]
    rec = datapoint[4]
    return valid_format.format(date=date,pos=pos,est_inf = est_inf,inc = inc,rec =rec)

def complete_data2xml(month_data):
    valid_root_format = """<?xml version="1.0" encoding="UTF-8"?>
<corona_data xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:noNamespaceSchemaLocation="Corona.xsd">
    {month_xml}
</corona_data>"""
    total_month_xml = ""
    for month in month_data:
        valid_month_format = """<month_data first_date = "{first_date}" last_date="{last_date}">
        {day_xml}
        </month_data>"""
        data = month_data[month]
        first_date = data[len(data)-1][0]
        last_date = data[0][0]
        total_day_xml = ""
        for datapoint in data:
            day_xml = daydata2xml(datapoint)
            total_day_xml = total_day_xml + day_xml + "\n"
        
        month_xml = valid_month_format.format(first_date = first_date, last_date = last_date, day_xml = total_day_xml)
        total_month_xml = total_month_xml + month_xml + "\n"
    return valid_root_format.format(month_xml = total_month_xml)


def save_parsed_xml_to_disk(month_data):
    with open('parsed_corona_data.xml', 'w') as output_file:
        output_file.write(complete_data2xml(month_data))


save_parsed_xml_to_disk(month_data)
